In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import collections

In [2]:
spark = SparkSession.builder.appName("DF-Ops").getOrCreate()

# Find total count of movie ratings

In [3]:
from pyspark.sql.types import StringType, DoubleType, IntegerType, StructType, StructField
schema = StructType([StructField('userId', IntegerType(), True),
                     StructField('movieId', IntegerType(), True),
                     StructField('rating', IntegerType(), True),
                     StructField('timestamp', DoubleType(), True)])

In [4]:
data = spark.read.csv('resources/u.data',sep = '\t', header = False, schema = schema)

In [5]:
data.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: double (nullable = true)



In [6]:
data.head(3)

[Row(userId=196, movieId=242, rating=3, timestamp=881250949.0),
 Row(userId=186, movieId=302, rating=3, timestamp=891717742.0),
 Row(userId=22, movieId=377, rating=1, timestamp=878887116.0)]

In [7]:
data.columns

['userId', 'movieId', 'rating', 'timestamp']

In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+-----------------+
|summary|            userId|           movieId|            rating|        timestamp|
+-------+------------------+------------------+------------------+-----------------+
|  count|            100000|            100000|            100000|           100000|
|   mean|         462.48475|         425.53013|           3.52986|8.8352885148862E8|
| stddev|266.61442012750905|330.79835632558473|1.1256735991443214|5343856.189502848|
|    min|                 1|                 1|                 1|      8.7472471E8|
|    max|               943|              1682|                 5|     8.93286638E8|
+-------+------------------+------------------+------------------+-----------------+



In [9]:
data.groupBy('rating').count().orderBy('rating').show()

+------+-----+
|rating|count|
+------+-----+
|     1| 6110|
|     2|11370|
|     3|27145|
|     4|34174|
|     5|21201|
+------+-----+



# Find average number of friends by age

In [10]:
# Load data
friendData = spark.read.csv("resources/fakefriends-header.csv", header = True, inferSchema = True)

In [11]:
friendData.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)



In [12]:
friendData.show(3)

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
|     2|    Hugh| 55|    221|
+------+--------+---+-------+
only showing top 3 rows



In [13]:
from pyspark.sql import functions as f
friendData.select('age','friends').groupBy('age').agg(f.round(f.avg('friends'),2).alias('AveFriends')).show()

+---+----------+
|age|AveFriends|
+---+----------+
| 31|    267.25|
| 65|     298.2|
| 53|    222.86|
| 34|     245.5|
| 28|     209.1|
| 26|    242.06|
| 27|    228.13|
| 44|    282.17|
| 22|    206.43|
| 47|    233.22|
| 52|    340.64|
| 40|    250.82|
| 20|     165.0|
| 57|    258.83|
| 54|    278.08|
| 48|     281.4|
| 19|    213.27|
| 64|    281.33|
| 41|    268.56|
| 43|    230.57|
+---+----------+
only showing top 20 rows



# Find min and max temperature in a year

In [14]:
from pyspark.sql.types import StringType, DoubleType, IntegerType, StructType, StructField
schema = StructType([StructField('stationId', StringType(), True),
                     StructField('date', IntegerType(), True),
                     StructField('entryType', StringType(), True),
                     StructField('temperature', DoubleType(), True),
                     StructField('column1', StringType(), True),
                     StructField('column2', StringType(), True),
                     StructField('column3', StringType(), True)])

In [15]:
data = spark.read.csv("resources/1800.csv",header = False,schema=schema)

In [16]:
data.printSchema()

root
 |-- stationId: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- entryType: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- column1: string (nullable = true)
 |-- column2: string (nullable = true)
 |-- column3: string (nullable = true)



In [17]:
data.show(3)

+-----------+--------+---------+-----------+-------+-------+-------+
|  stationId|    date|entryType|temperature|column1|column2|column3|
+-----------+--------+---------+-----------+-------+-------+-------+
|ITE00100554|18000101|     TMAX|      -75.0|   null|   null|      E|
|ITE00100554|18000101|     TMIN|     -148.0|   null|   null|      E|
|GM000010962|18000101|     PRCP|        0.0|   null|   null|      E|
+-----------+--------+---------+-----------+-------+-------+-------+
only showing top 3 rows



In [18]:
modifiedData = data.select('stationId','entryType','temperature')\
.withColumn('FTemperature',f.round((data.temperature * 0.1 * (9.0/5.0) + 32.0),2))
modifiedData.show(3)

+-----------+---------+-----------+------------+
|  stationId|entryType|temperature|FTemperature|
+-----------+---------+-----------+------------+
|ITE00100554|     TMAX|      -75.0|        18.5|
|ITE00100554|     TMIN|     -148.0|        5.36|
|GM000010962|     PRCP|        0.0|        32.0|
+-----------+---------+-----------+------------+
only showing top 3 rows



In [19]:
modifiedData.filter(data.entryType=='TMIN').groupBy('stationId').min('FTemperature').show()

+-----------+-----------------+
|  stationId|min(FTemperature)|
+-----------+-----------------+
|ITE00100554|             5.36|
|EZE00100082|              7.7|
+-----------+-----------------+



In [20]:
modifiedData.filter(data.entryType=='TMAX').groupBy('stationId').max('FTemperature')\
.select('stationId',f.col('max(FTemperature)').alias('Max_Temp')).show()

+-----------+--------+
|  stationId|Max_Temp|
+-----------+--------+
|ITE00100554|   90.14|
|EZE00100082|   90.14|
+-----------+--------+



# Count number of word occurrence

In [21]:
inputDF = spark.read.text("resources/book.txt")

In [22]:
words = inputDF.select(f.explode(f.split(inputDF.value,"\\W+")).alias('word'))
words.show(3)

+----------+
|      word|
+----------+
|      Self|
|Employment|
|  Building|
+----------+
only showing top 3 rows



In [23]:
words.filter(words.word !='')

DataFrame[word: string]

In [24]:
lowerCaseWords =  words.select(f.lower(words.word).alias('word'))
lowerCaseWords.show(3)

+----------+
|      word|
+----------+
|      self|
|employment|
|  building|
+----------+
only showing top 3 rows



In [25]:
wordCounts = lowerCaseWords.groupBy('word').count()
wordCounts.show(3)

+------+-----+
|  word|count|
+------+-----+
|online|   50|
|   few|   40|
|  some|  121|
+------+-----+
only showing top 3 rows



In [26]:
wordCountsSorted = wordCounts.sort('count', ascending=False)
wordCountsSorted.show(3)

+----+-----+
|word|count|
+----+-----+
| you| 1878|
|  to| 1828|
|your| 1420|
+----+-----+
only showing top 3 rows



In [27]:
wordCountsSorted.show(wordCountsSorted.count()) # To show all results

+--------------------+-----+
|                word|count|
+--------------------+-----+
|                 you| 1878|
|                  to| 1828|
|                your| 1420|
|                 the| 1292|
|                   a| 1191|
|                  of|  970|
|                 and|  934|
|                    |  772|
|                that|  747|
|                  it|  649|
|                  in|  616|
|                  is|  560|
|                 for|  537|
|                  on|  428|
|                 are|  424|
|                  if|  411|
|                   s|  391|
|                   i|  387|
|            business|  383|
|                 can|  376|
|                  be|  369|
|                  as|  343|
|                have|  321|
|                with|  315|
|                   t|  301|
|                this|  280|
|                  or|  278|
|                time|  255|
|                 but|  242|
|                they|  234|
|                will|  231|
|             

# Find the Total Amount Spent by Customer

In [28]:
schema = StructType([StructField('custId', IntegerType(), True),
                     StructField('itemId', IntegerType(), True),
                     StructField('cost', DoubleType(), True)])

In [29]:
data = spark.read.csv("resources/customer-orders.csv",header = False, schema = schema)

In [30]:
data.printSchema()

root
 |-- custId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- cost: double (nullable = true)



In [31]:
data.show(3)

+------+------+-----+
|custId|itemId| cost|
+------+------+-----+
|    44|  8602|37.19|
|    35|  5368|65.89|
|     2|  3391|40.64|
+------+------+-----+
only showing top 3 rows



In [32]:
data.groupBy('custId').sum('cost').orderBy('custId').show(data.count())

+------+------------------+
|custId|         sum(cost)|
+------+------------------+
|     0| 5524.949999999998|
|     1| 4958.600000000001|
|     2|           5994.59|
|     3|           4659.63|
|     4| 4815.050000000002|
|     5| 4561.069999999999|
|     6| 5397.879999999998|
|     7| 4755.070000000001|
|     8| 5517.240000000001|
|     9| 5322.649999999999|
|    10| 4819.700000000001|
|    11| 5152.290000000002|
|    12| 4664.589999999998|
|    13|           4367.62|
|    14| 4735.030000000001|
|    15| 5413.510000000001|
|    16|           4979.06|
|    17| 5032.679999999999|
|    18|           4921.27|
|    19|5059.4299999999985|
|    20| 4836.859999999999|
|    21|           4707.41|
|    22| 5019.449999999999|
|    23|4042.6499999999987|
|    24| 5259.920000000003|
|    25| 5057.610000000001|
|    26|            5250.4|
|    27| 4915.889999999999|
|    28| 5000.709999999998|
|    29| 5032.529999999999|
|    30|           4990.72|
|    31|           4765.05|
|    32| 5496.050000

In [33]:
spark.stop()